In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import warnings; warnings.filterwarnings("ignore")

final = -1

In [2]:
film_df = pd.read_csv("/tmp/movies.csv")


title = []
tahun = []

for t in film_df.iloc[:, 1]:
    split = t.split(" ")
    if len(split) == 1:
      title.append(split[0])
      tahun.append(np.nan)
    else:
      title.append(" ".join(split[:final]))
      tahun.append(split[final:][0][1:5])

film_df["Title"] = title
film_df["Tahun"] = tahun

film_df["Tahun"].fillna("2016", inplace=True)

replacer = {"Sci-Fi":"SciFi", "Film-Noir":"Filnoir", "|":" "}

for i, t in replacer.items():
    film_df["Genre"] = film_df["Genre"].str.replace(i, t)

ind_title = lambda i : film_df.iloc[i, 1]

In [3]:
user_df = pd.read_csv("/tmp/rating.csv")

user_df.isnull().sum().sum()

0

In [4]:
user_df.iloc[:5]

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
user_df.iloc[:, 0].nunique()

610

In [6]:
user_df.iloc[:, 1].nunique()

9724

In [7]:
user_df.loc[:, "rating"].min()

0.5

In [8]:
user_df.loc[:, "rating"].max()

5.0

In [9]:
user_df.loc[:, "rating"].mean().round(5)

3.50156

In [10]:
df = pd.pivot_table(user_df, index=user_df.columns[1], columns=user_df.columns[0],
                    values=user_df.columns[2], fill_value=0)

df.shape

(9724, 610)

In [11]:
df.iloc[:5, :15]

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
movieId,,,,,,,,,,,,,,,
1,4,0.0,0.0,0,4,0,4.5,0,0,0.0,0,0.0,0,0,2.5
2,0,0.0,0.0,0,0,4,0.0,4,0,0.0,0,0.0,0,0,0.0
3,4,0.0,0.0,0,0,5,0.0,0,0,0.0,0,0.0,0,0,0.0
4,0,0.0,0.0,0,0,3,0.0,0,0,0.0,0,0.0,0,3,0.0
5,0,0.0,0.0,0,0,5,0.0,0,0,0.0,0,0.0,0,0,0.0


In [12]:
ind_title(1)

'Jumanji'

```User```

In [13]:
inuser = user_df.groupby(user_df.columns[0])[[user_df.columns[2]]].agg("count")

inuser.index.names = ['User']

inuser = inuser.sort_values(by="rating", ascending=False)

inuser.iloc[:5]

,rating
User,
414,2698
599,2478
474,2108
448,1864
274,1346


In [14]:
inuser.tail()

,rating
User,
442,20
569,20
320,20
576,20
53,20


In [15]:
len(inuser)

610

In [16]:
len(user_df[user_df[user_df.columns[0]] == 414])

2698

In [17]:
len(df[df[414] != 0])

2698

In [18]:
limit = [0, 10, 50, 100, 500]

result = {}

for i in limit:
  result[i] = len(inuser[inuser["rating"] >= i])

result

{0: 610, 10: 610, 50: 385, 100: 248, 500: 44}

In [19]:
t = [col for col in df.columns if len(df[df[col] != 0]) >= 50]

len(t)

385

In [20]:
df = df.drop(labels=[col for col in df.columns if col not in t], axis=1)

len(df.columns)

385

```Film```

In [21]:
infilm = user_df.groupby(user_df.columns[1])[[user_df.columns[2]]].agg("count")

infilm.index.names = ['Film']

infilm = infilm.sort_values(by="rating", ascending=False)

infilm.iloc[:5]

,rating
Film,
356,329
318,317
296,307
593,279
2571,278


In [22]:
infilm.tail()

,rating
Film,
4093,1
4089,1
58351,1
4083,1
193609,1


In [23]:
len(infilm)

9724

In [24]:
len(user_df[user_df[user_df.columns[1]] == 318])

317

In [25]:
len(infilm[infilm["rating"] > 10])

2121

In [26]:
idlist = infilm[infilm["rating"] > 10].index.tolist()

df = df.drop(labels=idlist, axis=0)

len(df)

7603

In [27]:
df.shape

(7603, 385)